# Team Ernst Engel Nutritional Content Notebook
In this notebook, we obtain the nutritional contents for the foods of interest for our project. Moreover, we examine the nutritional adequacy of the diets of the housholds in our analysis.

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, widgets
import fooddatacentral as fdc

# This is a local file unique to Jordan's computer
# It contains his USDA FoodData Central API Key
# Keep Commented or it will cause an error
import config

## Getting Nutritional Contents

In [16]:
foods_oi = pd.read_csv('./data/foods_oi.csv').set_index('Food')
apikey = config.API_KEY
diet_refs = pd.read_csv('./data/us_diet_minimums.csv')
dr_categories = diet_refs['Nutrition'].values

In [18]:
def get_nutritional_content(foods_oi, apikey, dr_categories):
    '''
    Description
    --------------------------------------------------
    Gets the nutritional content for the food of 
    interest.
    
    Inputs
    --------------------------------------------------
    + foods_oi : pandas dataframe; foods of interest
        with their FDC id
    + apikey : string; FoodData Central API key
    + dr_categories : array; USDA dietary requirment
        categories
    
    Outputs
    --------------------------------------------------
    + food_nutrients : pandas dataframe; contains the
        nutritional contents of all foods of interest
        for the dr_categories
    '''
    food_nutrients = {}
    for food in foods_oi.index:
        try:
            FDC = foods_oi.loc[food,'FDC']
            food_nutrients[food] = fdc.nutrients(apikey,FDC).Quantity
        except:
            pass

    food_nutrients = pd.DataFrame(food_nutrients,dtype=float)

    food_nutrients = food_nutrients[food_nutrients.index.isin(dr_categories)].fillna(0)
    return food_nutrients

In [20]:
food_nutrients = get_nutritional_content(foods_oi, apikey, dr_categories)
food_nutrients

,Wheat Flour,Wheat Cereal,Corn Meal,Rolled Oats,Evaporated Milk,Cabbage,Potatoes,Spinach,Sweet Potatoes,Navy Beans,Sugar,Beets,Milk (Whole),Liver (Beef)
"Calcium, Ca",0.0,19.000,0.000,52.0,267.00,40.000,0.00,99.000,35.00,49.00,1.000,16.000,123.000,5.000
"Carbohydrate, by difference",70.7,76.300,78.790,67.5,10.00,5.800,16.89,3.630,20.00,16.13,99.770,9.560,4.630,3.890
Energy,345.0,359.000,345.000,375.0,133.00,103.000,74.00,97.000,82.00,97.00,1627.000,180.000,61.000,564.000
"Fiber, total dietary",2.6,10.400,0.000,10.0,0.00,2.500,2.00,2.200,3.50,4.00,0.000,2.800,0.000,0.000
"Folate, DFE",0.0,220.000,0.000,0.0,0.00,43.000,0.00,194.000,0.00,0.00,0.000,109.000,0.000,290.000
"Iron, Fe",0.0,27.600,0.000,5.0,0.00,0.470,0.73,2.710,0.59,1.61,0.060,0.800,0.000,4.900
"Magnesium, Mg",0.0,30.000,0.000,0.0,0.00,12.000,0.00,79.000,0.00,0.00,0.000,23.000,12.000,18.000
Niacin,0.0,7.250,5.455,0.0,0.00,0.234,0.00,0.724,0.00,0.00,0.000,0.334,0.105,13.175
"Phosphorus, P",0.0,112.000,0.000,0.0,0.00,26.000,0.00,49.000,0.00,0.00,0.000,40.000,101.000,387.000
"Potassium, K",0.0,126.000,167.000,352.0,300.00,170.000,439.00,558.000,341.00,323.00,2.000,325.000,150.000,313.000


## Examing Nutritional Adequacy

In [23]:
food_nutrients = pd.read_csv('./data/uganda_nutritional_contents.csv').set_index('n')
food_prices = pd.read_csv('./data/uganda_food_prices.csv').fillna(0)
hh_chars = pd.read_csv('./data/uganda_hh_chars.csv')
### Deals With Housholds from an  Unknown Region
hh_chars['m'] = hh_chars['m'].fillna('Unknown')
expenditures = pd.read_csv('./data/uganda_expenditures_19-20.csv').fillna(0)

In [25]:
year_range = '2019-20'
hhc_sub = hh_chars[hh_chars['t'] == year_range].reset_index(drop = True)
fp_sub = food_prices[food_prices['t'] == year_range].set_index('m').drop(columns = ['t'])
food_cols = expenditures.iloc[0:2, 3:].columns
fp_sub_avgs = fp_sub.reset_index(drop = True)
fp_sub_avgs = pd.DataFrame(fp_sub_avgs.mean()).rename(columns = {0 : 'Mean_Price'})

In [7]:
def get_col_counts(expenditures_df, food_col, prices, price_avs):
    counts = []
    for idx in expenditures_df.index:
        region = expenditures_df.loc[idx, 'm']
        expenditure = expenditures_df.loc[idx, food_col]
        if region == 'Unknown':
            # Imputes price from unknown region with the mean accross all regions
            price = fp_sub_avgs.loc[food_col][0]
            if price == 0: # Don't want to divide by zero
                count = 0
                counts.append(count)
            else:
                count = expenditure / price
                counts.append(count)   
        else:
            price = prices.loc[region, food_col]
            if price == 0: # Don't want to divide by zero
                count = 0
                counts.append(count)
            else:
                count = expenditure / price
                counts.append(count)
    return counts

In [8]:
def get_counts(expenditures_df, food_cols, prices, price_avs):
    count_df = expenditures_df.drop(columns = food_cols)
    for food_col in food_cols:
        counts = get_col_counts(expenditures_df, food_col, prices, price_avs)
        count_df[food_col] = counts
    return count_df

In [9]:
exp_counts = get_counts(expenditures, food_cols, fp_sub, fp_sub_avgs)

In [59]:
household_master = hhc_sub.merge(exp_counts, left_on = ['i', 't', 'm'], right_on = ['i', 't', 'm'])
household_master

,i,t,m,F 00-03,F 04-08,F 09-13,F 14-18,F 19-30,F 31-50,F 51+,...,Sugarcane,Sweet Bananas,Sweet Potatoes,Tea,Tomatoes,Waragi,Water,Wheat (flour),Yam,Yogurt
0,00c9353d8ebe42faabf5919b81d7fae7,2019-20,Eastern,1.0,0.0,0.0,3.0,1.0,0.0,1.0,...,0.0,0.000000,5.034965,0.020000,1.600000,0.0,0.0,0.0,0.865775,0.0
1,062da72d5d3a457e9336b62c8bb9096d,2019-20,Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
2,0d0e29faff394154a69562b4527b48b8,2019-20,Eastern,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,1.280000,0.0,0.0,0.0,0.000000,0.0
3,0e03e253c35d4333a1ffad2df9d38850,2019-20,Eastern,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.000000,6.293706,0.020000,0.800000,0.0,0.0,0.0,0.000000,0.0
4,1013000201,2019-20,Central,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.709975,0.000000,0.030769,3.126136,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3002,bfdf0d66403440ceab439b1e1c47cdea,2019-20,Eastern,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.000000,7.552448,0.010000,1.600000,0.0,0.0,0.0,0.288592,0.0
3003,c33f6cb57d9849949e08a7350dabb829,2019-20,Central,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.000000,1.290323,0.015385,1.488636,0.0,0.0,0.0,0.000000,0.0
3004,d10a687889de469687377204195f3db0,2019-20,Western,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.000000,2.720000,0.000000,1.333333,0.0,0.0,0.0,0.000000,0.0
3005,d24fa50d02c041969a42102d8ebdadc9,2019-20,Eastern,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.000000,37.762238,0.000000,1.920000,0.0,0.0,0.0,0.000000,0.0


In [11]:
hhc_sub_num = hhc_sub.set_index('i').iloc[:, 2:-1]
hhc_sub_num

,F 00-03,F 04-08,F 09-13,F 14-18,F 19-30,F 31-50,F 51+,M 00-03,M 04-08,M 09-13,M 14-18,M 19-30,M 31-50,M 51+
i,,,,,,,,,,,,,,
00c9353d8ebe42faabf5919b81d7fae7,1.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
062da72d5d3a457e9336b62c8bb9096d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
0d0e29faff394154a69562b4527b48b8,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
0e03e253c35d4333a1ffad2df9d38850,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1013000201,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c33f6cb57d9849949e08a7350dabb829,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d10a687889de469687377204195f3db0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
d24fa50d02c041969a42102d8ebdadc9,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [27]:
diet_refs_rev = diet_refs.set_index('Nutrition').drop(columns = ['Source']).rename(columns = {'C 1-3' : 'F 00-03', 
                                                                                              'F 4-8' : 'F 04-08',
                                                                                              'M 4-8' : 'M 04-08',
                                                                                              'F 9-13' : 'F 09-13',
                                                                                              'M 9-13' : 'M 09-13'})
diet_refs_rev['M 00-03'] = diet_refs_rev['F 00-03']
diet_refs_rev = diet_refs_rev[hhc_sub_num.columns]
diet_refs_rev

,F 00-03,F 04-08,F 09-13,F 14-18,F 19-30,F 31-50,F 51+,M 00-03,M 04-08,M 09-13,M 14-18,M 19-30,M 31-50,M 51+
Nutrition,,,,,,,,,,,,,,
Energy,1000.0,1200.0,1600.0,1800.0,2000.0,1800.0,1600.0,1000.0,1400.0,1800.0,2200.0,2400.0,2200.0,2000.0
Protein,13.0,19.0,34.0,46.0,46.0,46.0,46.0,13.0,19.0,34.0,52.0,56.0,56.0,56.0
"Fiber, total dietary",14.0,16.8,22.4,25.2,28.0,25.2,22.4,14.0,19.6,25.2,30.8,33.6,30.8,28.0
"Folate, DFE",150.0,200.0,300.0,400.0,400.0,400.0,400.0,150.0,200.0,300.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1300.0,1300.0,1000.0,1000.0,1200.0,700.0,1000.0,1300.0,1300.0,1000.0,1000.0,1000.0
"Carbohydrate, by difference",130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",7.0,10.0,8.0,15.0,18.0,18.0,8.0,7.0,10.0,8.0,11.0,8.0,8.0,8.0
"Magnesium, Mg",80.0,130.0,240.0,360.0,310.0,320.0,320.0,80.0,130.0,240.0,410.0,400.0,420.0,420.0
Niacin,6.0,8.0,12.0,14.0,14.0,14.0,14.0,6.0,8.0,12.0,16.0,16.0,16.0,16.0


In [28]:
diet_refs_rev.shape

(20, 14)

In [56]:
hh_diet_reqs = pd.DataFrame()
for i in hhc_sub_num.index:
    hh_char_vals = hhc_sub_num.loc[i].values
    row = [i]
    char_req_dot_prod = pd.DataFrame(hh_char_vals @ diet_refs_rev.T).T
    char_req_dot_prod.index = [i]
    hh_diet_reqs = pd.concat([hh_diet_reqs, char_req_dot_prod])

In [79]:
hh_consumption_cols = list(food_cols.to_numpy())
hh_consumption_cols.append('i')

hh_consumption = household_master[hh_consumption_cols].set_index('i')
hh_consumption

,Beans,Beef,Beer,Biscuits,Bongo,Bread,"Butter, etc.",Cabbages,Cake,Cassava,...,Sugarcane,Sweet Bananas,Sweet Potatoes,Tea,Tomatoes,Waragi,Water,Wheat (flour),Yam,Yogurt
i,,,,,,,,,,,,,,,,,,,,,
00c9353d8ebe42faabf5919b81d7fae7,1.582418,0.00,0.000000,0.0,0.0,0.000000,0.0,0.833636,0.0,4.857143,...,0.0,0.000000,5.034965,0.020000,1.600000,0.0,0.0,0.0,0.865775,0.0
062da72d5d3a457e9336b62c8bb9096d,0.000000,0.00,0.000000,0.0,0.0,0.111111,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
0d0e29faff394154a69562b4527b48b8,0.439560,0.45,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,3.238095,...,0.0,0.000000,0.000000,0.000000,1.280000,0.0,0.0,0.0,0.000000,0.0
0e03e253c35d4333a1ffad2df9d38850,1.230769,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,2.590476,...,0.0,0.000000,6.293706,0.020000,0.800000,0.0,0.0,0.0,0.000000,0.0
1013000201,0.000000,0.00,2.916667,0.0,0.0,0.000000,0.0,0.000000,0.0,2.857143,...,0.0,1.709975,0.000000,0.030769,3.126136,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bfdf0d66403440ceab439b1e1c47cdea,0.527473,1.00,0.000000,0.0,2.8,0.000000,0.0,0.297727,0.0,2.590476,...,0.0,0.000000,7.552448,0.010000,1.600000,0.0,0.0,0.0,0.288592,0.0
c33f6cb57d9849949e08a7350dabb829,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,1.785714,...,0.0,0.000000,1.290323,0.015385,1.488636,0.0,0.0,0.0,0.000000,0.0
d10a687889de469687377204195f3db0,0.909091,0.00,0.000000,0.0,0.0,0.000000,0.0,0.425325,0.0,0.000000,...,0.0,0.000000,2.720000,0.000000,1.333333,0.0,0.0,0.0,0.000000,0.0
